# Custom code for generating response functions & datasets:
- Currently, response functions are multi-dimensional sigmoids meaning all input-output relationships will be monotonic. Eventually, might be nice to support non-monotonic relationships as well, so that certain input features can have an "optimum" with worse performance on either side of the optimum.
- Also note: this currently only works for generating non-formulations datasets. Eventually, want to support formulations as well.

In [1]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Optional

## These functions are doing most of the work:

### TODO: allow user to add noise to the response functions (make use of the `noise` argument which currently does nothing)

In [40]:
### D-dimensional sigmoid function with the given set of D coefficients:
def sigmoid(input_row, coefs):
    value = 1 / (1 + np.exp(-1 * np.matmul(input_row, coefs)))
    return value


def build_sythetic_demo_dataset(inputs=5, outputs=1, num_rows=10, noise=0, coefs=None):

    ### TODO: allow user to add noise to the response functions (using the `noise` argument)
    
    if isinstance(inputs, int):
        num_inputs = inputs
    else:
        num_inputs = len(inputs)

    if isinstance(outputs, int):
        num_outputs = outputs
    else:
        num_outputs = len(outputs)

    
    # Allow user to set their own coefficients   
    if coefs==None:
        coefs = np.array([[np.random.uniform(-1, 1) for i in range(num_inputs)] for k in range(num_outputs)])

    
    X = np.array([[np.random.uniform(-2, 2) for i in range(num_inputs)] for j in range(num_rows)])

    y = list()
    for k in range(num_outputs):
        y.append(list())
        for row in X:
            y[k].append(sigmoid(row, coefs[k]))

    y = np.array(y)

    
    data_df = pd.DataFrame()

    for i in range(num_inputs):
        if isinstance(inputs, int):
            data_df[f"x_{i+1}"] = X[:, i]
        else:
            data_df[list(inputs)[i]] = X[:, i]
    
    for k in range(num_outputs):
        if isinstance(outputs, int):
            data_df[f"y_{k+1}"] = y[k]
        else:
            data_df[list(outputs)[k]] = y[k]

    
    return data_df, coefs

### TODO: Constrained Simplex Sampling

In [41]:
def sample_from_constrained_simplex(
    n_dimensions: int,
    constraints: Optional[List[Tuple[float, float]]] = None,
    max_attempts: int = 1000
):
    """
    Generate a random point from an N-dimensional simplex with optional element-wise constraints.
    
    Parameters:
        n_dimensions (int): Number of dimensions for the simplex
        constraints (List[Tuple[float, float]], optional): List of (min, max) constraints for each dimension.
            Use None for unconstrained dimensions. Example: [(0.2, 0.4), None, (0, 0.5)]
        max_attempts (int): Maximum number of attempts to find a valid solution
        
    Returns:
        numpy.ndarray: Array of N numbers between 0 and 1 that sum to 1 and satisfy constraints
        
    Raises:
        ValueError: If constraints are impossible to satisfy or if max_attempts is reached
    """
    # Initialize constraints if not provided
    if constraints is None:
        constraints = [None] * n_dimensions
    elif len(constraints) != n_dimensions:
        raise ValueError("Length of constraints must match n_dimensions")
    
    # Validate constraints
    total_min = sum(c[0] for c in constraints if c is not None)
    if total_min > 1:
        raise ValueError("Sum of minimum constraints exceeds 1")
    
    for attempt in range(max_attempts):
        try:
            # Generate initial random sample
            sample = np.random.random(n_dimensions)
            sample = sample / np.sum(sample)  # Normalize to sum to 1
            
            # Apply constraints iteratively
            for _ in range(n_dimensions * 2):  # Allow multiple passes for adjustment
                modified = False
                
                # Adjust values to meet constraints
                for i, constraint in enumerate(constraints):
                    if constraint is not None:
                        min_val, max_val = constraint
                        if sample[i] < min_val:
                            deficit = min_val - sample[i]
                            # Take deficit proportionally from unconstrained elements
                            free_indices = [j for j, c in enumerate(constraints) 
                                         if c is None or (j != i and sample[j] > c[0])]
                            if not free_indices:
                                raise ValueError("Cannot satisfy minimum constraint")
                            weights = np.array([sample[j] for j in free_indices])
                            weights = weights / weights.sum()
                            for j, w in zip(free_indices, weights):
                                sample[j] -= deficit * w
                            sample[i] = min_val
                            modified = True
                        elif sample[i] > max_val:
                            excess = sample[i] - max_val
                            # Distribute excess proportionally to unconstrained elements
                            free_indices = [j for j, c in enumerate(constraints) 
                                         if c is None or (j != i and sample[j] < c[1])]
                            if not free_indices:
                                raise ValueError("Cannot satisfy maximum constraint")
                            sample[free_indices] += excess / len(free_indices)
                            sample[i] = max_val
                            modified = True
                
                # Normalize to sum to 1
                sample = sample / np.sum(sample)
                
                # Check if all constraints are satisfied
                constraints_satisfied = all(
                    c is None or (c[0] <= v <= c[1])
                    for c, v in zip(constraints, sample)
                )
                
                if constraints_satisfied and abs(sum(sample) - 1.0) < 1e-10:
                    return sample
                
                if not modified:
                    break
                    
        except ValueError:
            continue
            
    raise ValueError(f"Could not find valid solution after {max_attempts} attempts")

### Convert ingredient recipe data tables from "Wide" to "Compact" format:

In [42]:
def wide_to_compact_format(df):
    """
    Convert formulation data from wide format to compact format.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in wide format where:
        - Each row is a formulation
        - Each column is an ingredient with its weight percentage
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in compact format with columns:
        - Ingredient A Name, Ingredient A weight %, Ingredient B Name, etc.
    """
    # Create an empty list to store the transformed rows
    compact_rows = []
    
    # Iterate through each formulation (row)
    for idx, row in df.iterrows():
        # Get non-zero ingredients and their percentages
        ingredients = row[row > 0]
        
        # Create a new row with alternating ingredient names and percentages
        new_row = {}
        for i, (ingredient_name, percentage) in enumerate(ingredients.items(), 1):
            new_row[f'Ingredient {chr(64+i)} Name'] = ingredient_name
            new_row[f'Ingredient {chr(64+i)} weight %'] = percentage
            
        compact_rows.append(new_row)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(compact_rows)
    
    return result_df

### Convert ingredient recipe data tables from "Compact" to "Wide" format:

In [43]:
def compact_to_wide_format(df):
    """
    Convert formulation data from compact format to wide format.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in compact format where:
        - Each row is a formulation
        - Columns alternate between ingredient names and weight percentages
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in wide format where:
        - Each row is a formulation
        - Each column is an ingredient with its weight percentage
    """
    # Create a list to store the transformed rows
    wide_rows = []
    
    # Get all unique ingredients across all formulations
    ingredient_columns = [col for col in df.columns if 'Name' in col]
    all_ingredients = set()
    for col in ingredient_columns:
        all_ingredients.update(df[col].dropna().unique())
    
    # Process each formulation
    for idx, row in df.iterrows():
        # Create a dictionary with all ingredients initialized to 0
        formulation = {ingredient: 0 for ingredient in all_ingredients}
        
        # Fill in the actual values
        for i in range(1, len(df.columns) // 2 + 1):
            name_col = f'Ingredient {chr(64+i)} Name'
            weight_col = f'Ingredient {chr(64+i)} weight %'
            
            if name_col in df.columns and pd.notna(row[name_col]):
                ingredient_name = row[name_col]
                formulation[ingredient_name] = row[weight_col]
        
        wide_rows.append(formulation)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(wide_rows)
    
    # Sort columns alphabetically for consistency
    result_df = result_df.reindex(sorted(result_df.columns), axis=1)
    
    return result_df

## Examples

### Example 1: generate arbitrary # of rows & columns, with no column names

In [44]:
data_df, coefs = build_sythetic_demo_dataset(inputs=9, outputs=4, num_rows=10)
data_df

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,y_1,y_2,y_3,y_4
0,-1.224826,-0.241741,0.952925,0.107281,0.934932,1.742672,1.592182,1.023009,0.966407,0.773325,0.253052,0.215506,0.909280
1,-0.280502,-0.682236,-1.580859,-1.232077,-0.049256,-1.258494,-1.720484,-0.894678,-0.827347,0.389284,0.956441,0.910037,0.028848
2,1.004654,-0.140498,-1.289267,1.301847,-0.536984,0.053219,0.562622,-0.904427,-0.535040,0.900912,0.549650,0.871829,0.754388
3,1.735865,-1.229554,-0.888718,-1.948611,0.922423,-1.511024,0.384055,-0.867925,-0.605369,0.392873,0.680362,0.790215,0.054621
4,0.980456,0.429023,-0.772173,1.831831,-1.631980,1.538150,-1.613531,1.067055,0.431481,0.744463,0.190336,0.813495,0.715627
5,1.467388,-0.770089,-0.370536,-0.795240,0.528731,0.366885,0.083699,-1.157520,0.733959,0.827688,0.358128,0.594077,0.420433
6,-1.520775,1.596422,0.661749,0.474628,1.094089,-1.641781,1.465489,-1.947737,0.551348,0.291852,0.761701,0.026655,0.882001
7,0.648988,0.362441,-1.410632,-0.467972,-0.338475,-0.346613,0.273624,-1.428930,1.230168,0.900076,0.358292,0.690616,0.677114
8,1.324818,-0.414858,-1.174103,0.696301,-0.896827,-1.755338,-1.831337,0.231407,-0.618406,0.162831,0.668418,0.866095,0.160854
9,-1.514401,-1.090046,-1.909057,-1.273769,0.048419,1.178897,-1.868462,-0.529677,-0.142304,0.900528,0.968644,0.963084,0.055920


In [45]:
coefs_df = pd.DataFrame(coefs)
coefs_df = coefs_df.rename(index={k: f"y_{k+1}" for k in range(len(coefs_df))})

### CHECK:
coefs_df

,0,1,2,3,4,5,6,7,8
y_1,-0.188553,-0.740929,-0.601486,0.074970,-0.871887,0.878894,0.696668,-0.790492,0.377941
y_2,-0.679077,-0.056262,-0.598784,0.116050,0.916881,-0.078870,-0.578552,-0.372129,-0.814906
y_3,-0.086770,-0.992336,-0.987376,-0.140167,-0.936866,0.196037,0.130129,0.026386,-0.395476
y_4,-0.248332,-0.320498,0.353619,0.923080,-0.696931,0.019569,0.896721,-0.224252,0.937884


### Example 2: create a laser welding dataset with named columns

#### Assign "reasonable" ranges and desired units for each input & output column:

In [46]:
inputs = {
    "general": {
        "Laser Power": {"min": 100, "max": 1000, "units": "W"},
        "Pulse Duration": {"min": 0.1, "max": 10, "units": "ms"},
        "Welding Speed": {"min": 1, "max": 200, "units": "mm/s"},
        "Beam Diameter": {"min": 0.1, "max": 3, "units": "mm"},
        "Focal Position": {"min": -2, "max": 5, "units": "mm"},
        # "Shielding Gas Type": {"min": , "max": , "units": "n/a"},  # leave out categorical inputs for now
        "Flow Rate": {"min": 5, "max": 25, "units": "L/min"},
        "Heat Input": {"min": 10, "max": 500, "units": "J/mm"},
        "Ambient Temperature": {"min": 20, "max": 30, "units": "degC"},
        "Cooling Rate": {"min": 10, "max": 1000, "units": "degC/s"},
    },
    "formulation": {
        "Ingredient A": {"min": 0.0, "max": 1.0, "units": ""},
        "Ingredient B": {"min": 0.1, "max": 0.4, "units": ""},
        "Ingredient C": {"min": 0.0, "max": 0.8, "units": ""},
    },
}

In [47]:
inputs = {
    "Laser Power": {"min": 100, "max": 1000, "units": "W"},
    "Pulse Duration": {"min": 0.1, "max": 10, "units": "ms"},
    "Welding Speed": {"min": 1, "max": 200, "units": "mm/s"},
    "Beam Diameter": {"min": 0.1, "max": 3, "units": "mm"},
    "Focal Position": {"min": -2, "max": 5, "units": "mm"},
    # "Shielding Gas Type": {"min": , "max": , "units": "n/a"},  # leave out categorical inputs for now
    "Flow Rate": {"min": 5, "max": 25, "units": "L/min"},
    "Heat Input": {"min": 10, "max": 500, "units": "J/mm"},
    "Ambient Temperature": {"min": 20, "max": 30, "units": "degC"},
    "Cooling Rate": {"min": 10, "max": 1000, "units": "degC/s"},
}

In [48]:
outputs = {
    "Hardness": {"min": 200, "max": 800, "units": "HV"},
    "Fatigue Life": {"min": 10000, "max": 100000, "units": "numCycles"},
    "Wear Rate": {"min": 0.01, "max": 1.0, "units": "mg/m"},
    "Cutting Efficiency": {"min": 0.1, "max": 5, "units": "m/s"},
}

In [49]:
data_df, coefs = build_sythetic_demo_dataset(inputs=inputs, outputs=outputs, num_rows=15)
data_df

,Laser Power,Pulse Duration,Welding Speed,Beam Diameter,Focal Position,Flow Rate,Heat Input,Ambient Temperature,Cooling Rate,Hardness,Fatigue Life,Wear Rate,Cutting Efficiency
0,0.556308,1.601968,1.893985,-1.638785,1.897153,1.791183,1.942661,-1.739371,0.518067,0.718289,0.077533,0.005744,0.025132
1,0.372488,1.205310,1.299496,-1.878383,-0.843209,-0.643894,1.576284,1.273039,-1.261175,0.140584,0.259280,0.100060,0.404936
2,-1.078284,1.510360,-1.933875,-0.016721,-0.494120,1.213839,-0.638682,-0.111126,-0.664243,0.878747,0.958655,0.020529,0.408665
3,0.358805,-1.606932,-0.407591,-1.616972,-0.793070,1.158591,-0.149575,0.114801,-0.718647,0.134263,0.279252,0.552968,0.914444
4,1.174504,-0.624797,0.428902,1.532930,0.970504,-0.507665,1.791624,0.520580,0.737501,0.650031,0.353507,0.664473,0.342163
5,1.005495,-0.306850,1.542144,-1.485175,-0.769597,1.596854,1.379755,-0.849200,0.548876,0.155002,0.284692,0.465769,0.470249
6,1.592803,1.557845,-1.580350,-0.888623,-0.514723,1.651529,-0.308785,0.782123,1.863760,0.944716,0.907309,0.088262,0.240410
7,-1.105011,0.585693,1.279932,-1.122642,-1.179470,-1.416760,-1.802204,-0.826002,1.204201,0.241727,0.460659,0.972304,0.344421
8,-0.259597,0.672356,-1.155267,1.815840,0.011168,1.659528,-1.455531,0.829989,0.142740,0.918282,0.973650,0.223461,0.634355
9,-0.206676,-1.775257,1.964440,1.187264,-1.054362,-0.347414,0.848473,0.935854,-0.214340,0.069867,0.611660,0.973091,0.885457


In [50]:
coefs_df = pd.DataFrame(coefs)

coefs_df = coefs_df.rename(columns={i: list(inputs)[i] for i in range(len(coefs_df.T))})
coefs_df = coefs_df.rename(index={k: list(outputs)[k] for k in range(len(coefs_df))})

### CHECK:
coefs_df

,Laser Power,Pulse Duration,Welding Speed,Beam Diameter,Focal Position,Flow Rate,Heat Input,Ambient Temperature,Cooling Rate
Hardness,-0.319518,0.599791,-0.636732,0.241493,0.676383,0.296027,0.060651,0.303258,0.677174
Fatigue Life,-0.769912,0.494003,-0.438935,0.723760,-0.884682,0.624275,0.151938,0.461919,0.475755
Wear Rate,0.819337,-0.998366,0.614139,0.585828,-0.991893,-0.956950,-0.870642,-0.519288,0.329193
Cutting Efficiency,0.177907,-0.841708,-0.040518,0.087411,-0.531787,0.163551,-0.369035,0.277397,-0.527780


## Re-scale each column to its desired range:

In [51]:
df = data_df.copy()

In [52]:
df_scaled = df.copy()

for col in df.columns:
    if col in inputs:
        scaled_col = (df[col].to_numpy() + 2) / 4
        df_scaled[col] = scaled_col

all_columns = dict()
all_columns.update(inputs)
all_columns.update(outputs)

for col in all_columns:
    df_scaled[col] = df_scaled[col] * (all_columns[col]["max"] - all_columns[col]["min"]) + all_columns[col]["min"]

column_renaming = {col: f'{col}_{all_columns[col]["units"]}' for col in all_columns}
df_scaled = df_scaled.rename(column_renaming, axis=1)

df_scaled

,Laser Power_W,Pulse Duration_ms,Welding Speed_mm/s,Beam Diameter_mm,Focal Position_mm,Flow Rate_L/min,Heat Input_J/mm,Ambient Temperature_degC,Cooling Rate_degC/s,Hardness_HV,Fatigue Life_numCycles,Wear Rate_mg/m,Cutting Efficiency_m/s
0,675.169248,9.014870,194.725745,0.361881,4.820019,23.955917,492.975937,20.651571,633.221503,630.973157,16977.938200,0.015686,0.223146
1,633.809902,8.033143,165.149920,0.188173,0.024385,11.780529,448.094799,28.182597,192.859217,284.350649,33335.182797,0.109059,2.084184
2,307.386040,8.788141,4.289740,1.537877,0.635290,21.069197,176.761413,24.722186,340.599776,727.248492,96278.910936,0.030323,2.102460
3,630.731162,1.072842,80.222353,0.377696,0.112127,20.792957,236.677038,25.287003,327.134806,280.557756,35132.722458,0.557438,4.580774
4,814.263493,3.503627,121.837870,2.661374,3.198382,12.461677,474.473972,26.301450,687.531461,590.018496,41815.585495,0.667828,1.776601
5,776.236392,4.290547,177.221653,0.473248,0.153206,22.984269,424.020016,22.877000,640.846786,293.000970,35622.240899,0.471112,2.404223
6,908.380731,8.905667,21.877605,0.905748,0.599235,23.257647,217.173850,26.955308,966.280671,766.829831,91657.812462,0.097379,1.278008
7,301.372482,6.499591,164.176605,0.736085,-0.564073,7.916201,34.230035,22.934995,803.039701,345.036056,51459.277742,0.972581,1.787661
8,491.590738,6.714082,43.025483,2.866484,1.519545,23.297640,76.697447,27.074973,540.328092,750.969027,97628.534864,0.231226,3.208338
9,503.498010,0.656239,198.230879,2.410767,-0.345134,13.262930,358.937943,27.339634,451.950917,241.920490,65049.434772,0.973360,4.438738


## [Optional] Save result to Excel or CSV file: 

In [53]:
# df_scaled.to_excel("Demo Datasets/Laser Welding (Synthetic)/laser_welding.xlsx", index=False)
# df_scaled.to_csv("Demo Datasets/Laser Welding (Synthetic)/laser_welding.csv", index=False)

# Done!

# SCRATCH

## TODO: Simplex Sampling

In [54]:
def sample_from_simplex(n_dimensions):
    """
    Generate a random point from an N-dimensional simplex.
    
    The generated point will have N elements between 0 and 1 that sum to 1.
    This implementation uses the stick-breaking construction method.
    
    Parameters:
        n_dimensions (int): Number of dimensions for the simplex
        
    Returns:
        numpy.ndarray: Array of N numbers between 0 and 1 that sum to 1
    """
    # Generate N-1 random numbers between 0 and 1
    breaks = np.random.random(n_dimensions - 1)
    breaks.sort()  # Sort to implement stick-breaking construction
    
    # Add boundary points
    points = np.zeros(n_dimensions + 1)
    points[0] = 0
    points[1:-1] = breaks
    points[-1] = 1
    
    # Take differences to get segment lengths
    samples = np.diff(points)
    
    # Verify results
    assert np.all(samples >= 0)  # All values should be non-negative
    assert np.abs(np.sum(samples) - 1.0) < 1e-10  # Sum should be 1
    
    return samples

# Example usage
def demo_simplex_sampling():
    # Generate and test a few samples
    for dims in [2, 3, 4]:
        sample = sample_from_simplex(dims)
        print(f"\n{dims}-dimensional sample:")
        print(f"Values: {sample}")
        print(f"Sum: {np.sum(sample)}")

### TEST
demo_simplex_sampling()


2-dimensional sample:
Values: [0.22188868 0.77811132]
Sum: 1.0

3-dimensional sample:
Values: [0.07124603 0.23368628 0.69506769]
Sum: 1.0

4-dimensional sample:
Values: [0.19609372 0.20606278 0.59006293 0.00778057]
Sum: 1.0


In [55]:
sample_from_simplex(5)

array([0.26394811, 0.47895778, 0.18299992, 0.01113522, 0.06295899])

### TODO: Constrained Simplex Sampling

In [56]:
# Example usage and testing function
def demo_constrained_sampling():
    # Test case 1: Constraint on first element
    constraints = [(0.2, 0.4), None, None]
    sample = sample_from_constrained_simplex(3, constraints)
    print("\nTest case 1 - First element constrained to [0.2, 0.4]:")
    print(f"Values: {sample}")
    print(f"Sum: {np.sum(sample)}")
    print(f"First element in range: {0.2 <= sample[0] <= 0.4}")
    
    # Test case 2: Multiple constraints
    constraints = [(0.2, 0.4), (0, 0.3), None]
    sample = sample_from_constrained_simplex(3, constraints)
    print("\nTest case 2 - Multiple constraints:")
    print(f"Values: {sample}")
    print(f"Sum: {np.sum(sample)}")
    print(f"Constraints satisfied: {all([0.2 <= sample[0] <= 0.4, 0 <= sample[1] <= 0.3])}")


### TEST
demo_constrained_sampling()


Test case 1 - First element constrained to [0.2, 0.4]:
Values: [0.2        0.54396326 0.25603674]
Sum: 1.0000000000000002
First element in range: True

Test case 2 - Multiple constraints:
Values: [0.4        0.29797349 0.30202651]
Sum: 0.9999999999999999
Constraints satisfied: True


In [57]:
constraints = [(0.2, 0.4), None, None]
sample_from_constrained_simplex(n_dimensions=3, constraints=constraints)

array([0.4      , 0.2892729, 0.3107271])

In [58]:
constraints = [(0.2, 0.4), None, None, (0.01, 0.05), None]
sample = sample_from_constrained_simplex(n_dimensions=5, constraints=constraints)
print(sample.sum())
sample.tolist()

1.0


[0.22023684477006109,
 0.2917388517753951,
 0.17166965446776308,
 0.05,
 0.26635464898678074]

## Convert ingredient recipe data tables from "Wide" to "Compact" format:

In [59]:
# Example usage

# Create sample data in wide format
wide_data = {
    'Sugar': [10, 0, 15, 0],
    'Salt': [2, 1, 0, 0],
    'Flour': [83, 85, 73, 73],
    'Baking Powder': [0, 2, 5, 5],
    'Vanilla': [0, 7, 0, 0],
    'Brown Sugar': [0, 0, 0, 15],
    'Milk Chocolate Chips': [5, 0, 0, 0],
    'Dark Chocolate Chips': [0, 0, 5, 0],
    'White Chocolate Chips': [0, 0, 0, 5],
}

wide_df = pd.DataFrame(wide_data)

In [60]:
print("Original wide format:")
wide_df

Original wide format:


,Sugar,Salt,Flour,Baking Powder,Vanilla,Brown Sugar,Milk Chocolate Chips,Dark Chocolate Chips,White Chocolate Chips
0,10,2,83,0,0,0,5,0,0
1,0,1,85,2,7,0,0,0,0
2,15,0,73,5,0,0,0,5,0
3,0,0,73,5,0,15,0,0,5


In [61]:
print("\nTransformed compact format:")
wide_to_compact_format(wide_df)


Transformed compact format:


,Ingredient A Name,Ingredient A weight %,Ingredient B Name,Ingredient B weight %,Ingredient C Name,Ingredient C weight %,Ingredient D Name,Ingredient D weight %
0,Sugar,10,Salt,2,Flour,83,Milk Chocolate Chips,5
1,Salt,1,Flour,85,Baking Powder,2,Vanilla,7
2,Sugar,15,Flour,73,Baking Powder,5,Dark Chocolate Chips,5
3,Flour,73,Baking Powder,5,Brown Sugar,15,White Chocolate Chips,5


## Convert ingredient recipe data tables from "Compact" to "Wide" format:

In [62]:
def compact_to_wide_format(df):
    """
    Convert formulation data from compact format to wide format.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in compact format where:
        - Each row is a formulation
        - Columns alternate between ingredient names and weight percentages
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in wide format where:
        - Each row is a formulation
        - Each column is an ingredient with its weight percentage
    """
    # Create a list to store the transformed rows
    wide_rows = []
    
    # Get all unique ingredients across all formulations
    ingredient_columns = [col for col in df.columns if 'Name' in col]
    all_ingredients = set()
    for col in ingredient_columns:
        all_ingredients.update(df[col].dropna().unique())
    
    # Process each formulation
    for idx, row in df.iterrows():
        # Create a dictionary with all ingredients initialized to 0
        formulation = {ingredient: 0 for ingredient in all_ingredients}
        
        # Fill in the actual values
        for i in range(1, len(df.columns) // 2 + 1):
            name_col = f'Ingredient {chr(64+i)} Name'
            weight_col = f'Ingredient {chr(64+i)} weight %'
            
            if name_col in df.columns and pd.notna(row[name_col]):
                ingredient_name = row[name_col]
                formulation[ingredient_name] = row[weight_col]
        
        wide_rows.append(formulation)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(wide_rows)
    
    # Sort columns alphabetically for consistency
    result_df = result_df.reindex(sorted(result_df.columns), axis=1)
    
    return result_df

In [63]:
wide_to_compact_format(wide_df)

,Ingredient A Name,Ingredient A weight %,Ingredient B Name,Ingredient B weight %,Ingredient C Name,Ingredient C weight %,Ingredient D Name,Ingredient D weight %
0,Sugar,10,Salt,2,Flour,83,Milk Chocolate Chips,5
1,Salt,1,Flour,85,Baking Powder,2,Vanilla,7
2,Sugar,15,Flour,73,Baking Powder,5,Dark Chocolate Chips,5
3,Flour,73,Baking Powder,5,Brown Sugar,15,White Chocolate Chips,5


In [64]:
compact_to_wide_format(wide_to_compact_format(wide_df))

,Baking Powder,Brown Sugar,Dark Chocolate Chips,Flour,Milk Chocolate Chips,Salt,Sugar,Vanilla,White Chocolate Chips
0,0,0,0,83,5,2,10,0,0
1,2,0,0,85,0,1,0,7,0
2,5,0,5,73,0,0,15,0,0
3,5,15,0,73,0,0,0,0,5


In [65]:
wide_df[sorted(wide_df.columns)]

,Baking Powder,Brown Sugar,Dark Chocolate Chips,Flour,Milk Chocolate Chips,Salt,Sugar,Vanilla,White Chocolate Chips
0,0,0,0,83,5,2,10,0,0
1,2,0,0,85,0,1,0,7,0
2,5,0,5,73,0,0,15,0,0
3,5,15,0,73,0,0,0,0,5
